In [18]:
## 참고: 한 권으로 끝내는 실전 LLM 파인튜닝
## 2장 GPT
## 2.1 런팟 개발 환경 설정
## 2.2 데이터 준비와 모델 구성
%pip install -q datasets


[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
from datasets import load_dataset

dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")
data = dataset
data

README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/8.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [20]:
data['train']['document'][0]

'앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.'

In [22]:
ko_text = "".join(data["train"]["document"])
ko_chars = sorted(list(set((ko_text))))  # ['왓', '왔', '왕', '왜', '왠', ... ] 이런식으로 담김
ko_vocab_size = len(ko_chars)
print(ko_vocab_size)  # 총 글자 수

2701


In [24]:
# 문자와 인덱스를 매핑하는 딕셔너리 생성
character_to_ids = {char:i for i, char in enumerate(ko_chars)}  # char -> i
ids_to_character = {i:char for i, char in enumerate(ko_chars)}  # i -> char

token_encode = lambda s:[character_to_ids[c] for c in s]  # 문자열 s의 각 문자 c를 순회하며 character_to_ids 사전을 활용해 해당 문자를 숫자로 변환
token_decode = lambda l: "".join([ids_to_character[i] for i in l])  # 리스트 l의 숫자 i를 순회하며 ids_to_character 사전을 활용해 해당 숫자에 맞는 문자로 변환
print(token_encode("안녕하세요 함께 인공지능을 공부하게 되어 반가워요."))
print(token_decode(token_encode("안녕하세요 함께 인공지능을 공부하게 되어 반가워요.")))

[1909, 1169, 2546, 1770, 2008, 0, 2551, 1061, 0, 2064, 977, 2157, 1209, 2055, 0, 977, 1658, 2546, 949, 0, 1283, 1942, 0, 1593, 908, 2024, 2008, 2]
안녕하세요 함께 인공지능을 공부하게 되어 반가워요.


In [25]:
import torch

tokenized_data = torch.tensor(token_encode(ko_text), dtype=torch.long)
print(tokenized_data.shape, tokenized_data.dtype)
print(tokenized_data[:100])

torch.Size([22162967]) torch.int64
tensor([1928, 2315,    0, 2105, 1658,  908,    0, 1987, 2555,    0, 2546, 1593,
        1028,    0, 2015, 1485,    0,  965, 2107, 2060,    0, 1617, 2465, 1542,
        2064,    0, 1808, 2273,    0, 2603, 1236, 1477,    0, 2037, 2555,    0,
        2263, 1430, 2055,    0, 1028, 2019, 2062, 1028, 1441,    0, 2562, 1841,
        1213, 1221,    2,    0, 2451, 2650,    0, 1808, 2273,    0, 2142, 1787,
        1028, 1950, 2060,    0, 1558, 1468, 1119,    0, 2555, 1787, 1477,    0,
        2037, 2555,    0, 1553, 1967, 1024, 2051,    0, 1015, 1541, 1477,    0,
           7,    3, 2117,    0, 2026,    0, 2062, 1740,    0, 2603, 1236, 2546,
         968,    0, 1558, 1468])


In [26]:
# train 데이터와 val 데이터로 나누기
n = int(0.9 * len(tokenized_data))
train_dataset = tokenized_data[:n]
test_dataset = tokenized_data[n:]

In [27]:
block_size = 8
train_dataset[:block_size]

tensor([1928, 2315,    0, 2105, 1658,  908,    0, 1987])

In [28]:
x = train_dataset[:block_size]
y = train_dataset[1:block_size + 1]

for time in range(block_size):
    context = x[:time + 1]
    target = y[time]

print(f"입력 텐서: {context}")
print(f"타깃 글자: {target}")

# block_size는 각 텐서의 길이를 결정하고
# batch_size는 한번에 처리할 텐서의 갯수를 설정

입력 텐서: tensor([1928, 2315,    0, 2105, 1658,  908,    0, 1987])
타깃 글자: 2555


In [29]:
torch.manual_seed(1234)

batch_size = 4
block_size = 8

def batch_function(mode):
    dataset = train_dataset if mode == "train" else test_dataset
    idx = torch.randint(len(dataset) - block_size, (batch_size,))
    x = torch.stack([dataset[index:index + block_size] for index in idx])
    y = torch.stack([dataset[index + 1:index + block_size + 1] for index in idx])
    return x, y

example_x, example_y = batch_function("train")
print("inputs: ", example_x.shape)  # torch.Size([배치 사이즈, 블록 사이즈])
print("")
print("example_x의 실제 값")
print(example_x)
print("=================")

print("targets: ", example_y.shape)
print("")
print("example_y의 실제 값")
print(example_y)
print("=================")

for size in range(batch_size):
    for t in range(block_size):
        context = example_x[size, :t+1]
        target = example_y[size, t]
        print(f"input: {context}, target: {target}")
    print("=================")
    print("=================")

# 모델이 각 배치 시퀀스를 처리하며 텍스트 구조와 언어 패턴을 학습하는 과정

inputs:  torch.Size([4, 8])

example_x의 실제 값
tensor([[1764, 2555,    0, 1236, 2248,    0, 2017, 1976],
        [   0, 1966, 2157,    0, 1951, 2062,    0, 2548],
        [   0, 1304, 1485, 1586,    0, 1907, 2450,    0],
        [   3,    2,    6,    5,    1,    0,    5,    3]])
targets:  torch.Size([4, 8])

example_y의 실제 값
tensor([[2555,    0, 1236, 2248,    0, 2017, 1976, 2546],
        [1966, 2157,    0, 1951, 2062,    0, 2548, 2289],
        [1304, 1485, 1586,    0, 1907, 2450,    0, 2480],
        [   2,    6,    5,    1,    0,    5,    3,    5]])
input: tensor([1764]), target: 2555
input: tensor([1764, 2555]), target: 0
input: tensor([1764, 2555,    0]), target: 1236
input: tensor([1764, 2555,    0, 1236]), target: 2248
input: tensor([1764, 2555,    0, 1236, 2248]), target: 0
input: tensor([1764, 2555,    0, 1236, 2248,    0]), target: 2017
input: tensor([1764, 2555,    0, 1236, 2248,    0, 2017]), target: 1976
input: tensor([1764, 2555,    0, 1236, 2248,    0, 2017, 1976]), target

In [30]:
## 2.3 언어 모델 만들기
import torch
import torch.nn as nn
from torch.nn import functional as F

class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)
        # vocab_length은 모델이 다룰 수 있는 단어의 총 개수: 2701, 첫번째 vocab_length는 총 단어의 수, 두번째 vocab_length는 각 단어를 표현할 벡터의 크기

    def forward(self, inputs, targets):
        logits = self.embedding_token_table(inputs)

        return logits

model = semiGPT(ko_vocab_size)
output = model(example_x, example_y)
print(output.shape)

torch.Size([4, 8, 2701])


In [31]:
## forward 메서드
# 에러 발생 코드  # shape 불일치에 관한 실수
import torch
import torch.nn as nn
from torch.nn import functional as F

class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs, targets):
        logits = self.embedding_token_table(inputs)

        loss = F.cross_entropy(logits, targets)
        return logits, loss

model = semiGPT(ko_vocab_size)
output, loss = model(example_x, example_y)
print(output)  # RuntimeError: Expected target size [4, 2701], got [4, 8]  # 모델은 target size [4, 2701]을 기대했지만, 실제는 got [4, 8]을 받았다.

RuntimeError: Expected target size [4, 2701], got [4, 8]

In [32]:
# => logits의 shape를 [4, 8, 2701]에서 [32, 2701]로 변경
# => targets의 shape를 [4, 8]에서 [32]로 변경
# 코드 수정
import torch
import torch.nn as nn
from torch.nn import functional as F

class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs, targets):
        logits = self.embedding_token_table(inputs)
        batch, seq_length, vocab_length = logits.shape
        
        logits = logits.view(batch * seq_length, vocab_length)  # logits의 shape 변경
        targets = targets.view(batch * seq_length)              # target의 shape 변경
        loss = F.cross_entropy(logits, targets)

        print("logits의 shape는: ", logits.shape, "입니다.")
        print("targets의 shape는: ", targets.shape, "입니다.")

        return logits, loss

model = semiGPT(ko_vocab_size)
output, loss = model(example_x, example_y)
print(loss)

logits의 shape는:  torch.Size([32, 2701]) 입니다.
targets의 shape는:  torch.Size([32]) 입니다.
tensor(8.5369, grad_fn=<NllLossBackward0>)


In [33]:
## generate 메서드 추가
import torch
import torch.nn as nn
from torch.nn import functional as F

class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs, targets=None):
        logits = self.embedding_token_table(inputs)
        
        if targets is None:  # 만약에 target이 없으면 loss도 없음
            loss = None
            
        else:
            batch, seq_length, vocab_length = logits.shape
            logits = logits.view(batch * seq_length, vocab_length)
            targets = targets.view(batch*seq_length)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(inputs)
            logits = logits[:, -1, :]  # 두번째 차원의 단일 시퀀스 선택으로 인해 결과 텐서의 형태는 (1, 4), 2차원 텐서 => [1, 2701] 형태로 shape을 변경
            # print(logits.shape)
            probs = F.softmax(logits, dim=-1)
            next_inputs = torch.multinomial(probs, num_samples=1)
            inputs = torch.cat((inputs, next_inputs), dim=1)

        return inputs

model = semiGPT(ko_vocab_size)
output, loss = model(example_x, example_y)
print(loss)

token_decode(model.generate(torch.zeros((1, 1),
                            dtype=torch.long),
                            max_new_tokens=10)[0].tolist())

tensor(8.5844, grad_fn=<NllLossBackward0>)


' 렌起탸三튕욥팬上＂빕'

In [34]:
## 옵티마이저 추가
learning_rate = 1e-2
model = semiGPT(ko_vocab_size)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [35]:
from tqdm.auto import tqdm

batch_size = 32

for steps in tqdm(range(10000)):
    example_x, example_y = batch_function("train")
    logits, loss = model(example_x, example_y)
    # 옵티마이저 초기화
    optimizer.zero_grad(set_to_none=True)
    # 역전파 계산
    loss.backward()
    # 가중치 업데이트
    optimizer.step()

print(loss.item())

  0%|          | 0/10000 [00:00<?, ?it/s]

3.6928412914276123


In [37]:
print(token_decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=10)[0].tolist()))

 공편지로 살펴보험을


In [38]:
## 데이터를 GPU로 전달하기
device = "cuda" if torch.cuda.is_available() else "cpu"

In [39]:
def batch_function(mode):
    dataset = train_dataset if mode == "train" else test_dataset
    idx = torch.randint(len(dataset) - block_size, (batch_size,))
    x = torch.stack([dataset[index:index+block_size] for index in idx])
    y = torch.stack([dataset[index+1:index+block_size+1] for index in idx])
    x, y = x.to(device), y.to(device) # .to 를 추가
    return x, y

In [40]:
## loss 함수 생성
@torch.no_grad()
def compute_loss_metrics():
    out = {}
    model.eval()  # .eval(): 학습 과정과 구분되는 평가 모드 ex) 학습시에는 드롭아웃으로 일부 뉴런을 비활성화 하지만, 평가시에는 모든 뉴런을 활성화하여 안정적인 예측을 수행
    for mode in ["trian", "eval"]:
        losses = torch.zeros(eval_iteration)
        for k in range(eval_iteration):
            inputs, targets = batch_function(mode)
            logits, loss = model(inputs, targets)
            losses[k] = loss.item()
        out[mode] = losses.mean()
    model.train()
    return out

In [41]:
## 전체 코드 복습
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size = 32
block_size = 8
max_iteration = 50000
eval_interval = 300
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iteration = 200

def batch_function(mode):
    dataset = train_dataset if mode == "train" else test_dataset
    idx = torch.randint(len(dataset) - block_size, (batch_size,))
    x = torch.stack([dataset[index:index+block_size] for index in idx])
    y = torch.stack([dataset[index+1:index+block_size+1] for index in idx])
    x, y = x.to(device), y.to(device) # .to 를 추가
    return x, y

@torch.no_grad()
def compute_loss_metrics():
    out = {}
    model.eval()
    for mode in ["train", "eval"]:
        losses = torch.zeros(eval_iteration)
        for k in range(eval_iteration):
            inputs, targets = batch_function(mode)
            logits, loss = model(inputs, targets)
            losses[k] = loss.item()
        out[mode] = losses.mean()
    model.train()
    return out

class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs, targets=None):
        logits = self.embedding_token_table(inputs)
        if targets is None:
            loss = None
        else:
            batch, seq_length, vocab_length = logits.shape
            logits = logits.view(batch * seq_length, vocab_length)
            targets = targets.view(batch*seq_length)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(inputs)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_inputs = torch.multinomial(probs, num_samples=1)
            inputs = torch.cat((inputs, next_inputs), dim=1)
        return inputs

model = semiGPT(ko_vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for step in range(max_iteration):
    if step % eval_interval == 0 :
        losses = compute_loss_metrics()
        print(f'step : {step}, train loss : {losses["train"]:.4f}, val loss : {losses["eval"]:.4f}')

    example_x, example_y = batch_function("train")
    logits, loss = model(example_x, example_y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

inputs = torch.zeros((1,1), dtype=torch.long, device=device)
print(token_decode(model.generate(inputs, max_new_tokens=100)[0].tolist()))

step : 0, train loss : 8.4359, val loss : 8.4330
step : 300, train loss : 6.1704, val loss : 6.1665
step : 600, train loss : 4.8009, val loss : 4.7797
step : 900, train loss : 4.2406, val loss : 4.2343
step : 1200, train loss : 3.9565, val loss : 3.9551
step : 1500, train loss : 3.8141, val loss : 3.8070
step : 1800, train loss : 3.7212, val loss : 3.7257
step : 2100, train loss : 3.6543, val loss : 3.6675
step : 2400, train loss : 3.6260, val loss : 3.6315
step : 2700, train loss : 3.5858, val loss : 3.5753
step : 3000, train loss : 3.5355, val loss : 3.5677
step : 3300, train loss : 3.5347, val loss : 3.5463
step : 3600, train loss : 3.5198, val loss : 3.5005
step : 3900, train loss : 3.5033, val loss : 3.5166
step : 4200, train loss : 3.4893, val loss : 3.5015
step : 4500, train loss : 3.4775, val loss : 3.4725
step : 4800, train loss : 3.4721, val loss : 3.4741
step : 5100, train loss : 3.4353, val loss : 3.4628
step : 5400, train loss : 3.4499, val loss : 3.4723
step : 5700, train

In [42]:
## 셀프 어텐션 추가하기
import torch
torch.manual_seed(1441)
num_batches, sequence_length, embedding_dim = 2, 4, 6
embeddings_tensor = torch.randn(num_batches,
                                sequence_length,
                                embedding_dim)
embeddings_tensor.shape

torch.Size([2, 4, 6])

In [43]:
# 이전 임베딩의 평균을 저장할 텐서 초기화
averaged_embeddings = torch.zeros((num_batches, sequence_length, embedding_dim))

# 각 배치에 대해 반복
for batch_index in range(num_batches):
    # 각 시퀀스 위치에 대해 반복
    for sequence_position in range(sequence_length):
        # 현재 시퀀스 위치까지의 이전 임베딩을 슬라이스
        previous_embeddings = embeddings_tensor[batch_index, :sequence_position + 1]
        # 현재 위치까지의 임베딩의 평균을 계산
        averaged_embeddings[batch_index, sequence_position] = torch.mean(previous_embeddings, dim=0)

print(embeddings_tensor[0])
print(averaged_embeddings[0])

tensor([[-1.1437, -1.2611, -0.1634, -0.5255, -1.0879,  0.3712],
        [ 2.2335,  0.3099, -1.3975,  1.1141, -0.3373,  0.6924],
        [ 0.2644,  1.1567, -0.5040, -0.7986,  2.6778,  1.4161],
        [ 1.3159, -0.5231,  1.2933, -0.8819,  0.7118,  0.4209]])
tensor([[-1.1437, -1.2611, -0.1634, -0.5255, -1.0879,  0.3712],
        [ 0.5449, -0.4756, -0.7804,  0.2943, -0.7126,  0.5318],
        [ 0.4514,  0.0685, -0.6883, -0.0700,  0.4175,  0.8266],
        [ 0.6675, -0.0794, -0.1929, -0.2730,  0.4911,  0.7252]])


In [44]:
## 행렬곱 연산으로 더 빠르게 정보 주고받기
# 행렬곱 연산 예시
A = torch.ones(3, 3)
B = torch.randint(0, 10, (3, 2)).float()
AB = A @ B

print("A 행렬")
print(A)
print("=================")
print("=================")
print("B 행렬")
print(B)
print("=================")
print("=================")
print("AB 행렬")
print(AB)

A 행렬
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
B 행렬
tensor([[7., 2.],
        [0., 5.],
        [2., 2.]])
AB 행렬
tensor([[9., 9.],
        [9., 9.],
        [9., 9.]])


In [45]:
weight = torch.tril(torch.ones(sequence_length, sequence_length))
print(weight)
weight = weight / weight.sum(1, keepdim=True)
print(weight)

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])


In [46]:
matirx_averaged_embeddings = weight @ embeddings_tensor
torch.allclose(averaged_embeddings, matirx_averaged_embeddings)

True

In [47]:
weight = torch.tril(torch.ones(sequence_length, sequence_length))
weight = weight.masked_fill(weight == 0, float('-inf'))  # 0 값을 -inf로 마스킹
print(weight)
weight = F.softmax(weight, dim=-1)
print(weight)

tensor([[1., -inf, -inf, -inf],
        [1., 1., -inf, -inf],
        [1., 1., 1., -inf],
        [1., 1., 1., 1.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])


In [48]:
weight_tril_embeddings = weight @ embeddings_tensor
torch.allclose(averaged_embeddings, weight_tril_embeddings)

True

In [50]:
## 셀프 어텐션
import torch
import torch.nn as nn
import torch.nn.functional as F

# 난수 고정
torch.manual_seed(1111)

# 배치 크기, 시퀀스 길이, 채널 수 설정
batch_size, seq_length, num_channels = 2, 4, 4
input_tensor = torch.randn(batch_size, seq_length, num_channels)

# 각 헤드의 크기
head_size = 16

# Key, Query, Value 변환을 위한 선형 레이어
key_transform = nn.Linear(num_channels, head_size, bias=False)
query_transform = nn.Linear(num_channels, head_size, bias=False)
value_transform = nn.Linear(num_channels, head_size, bias=False)

# Key, Query, Value 변환 수행
keys = key_transform(input_tensor)
queries = query_transform(input_tensor)
values = value_transform(input_tensor)

# 어텐션 스코어 계산
attention_scores = queries @ keys.transpose(-2, -1)

# 하삼각행렬 생성 및 마스킹
mask_lower_triangle = torch.tril(torch.ones(seq_length, seq_length))
attention_scores = attention_scores.masked_fill(mask_lower_triangle == 0, float('-inf'))

# 소프트맥스 함수를 사용해 확률 정규화
normalized_scores = F.softmax(attention_scores, dim=-1)

# 최종 출력 계싼
output_tensor = normalized_scores @ values

output_tensor

tensor([[[-0.4755, -0.5409, -0.1864,  0.2951, -1.0717, -0.6172, -0.0176,
           0.1793, -0.1113,  0.6589, -0.4507, -0.1181, -0.9728, -0.8870,
           0.2349, -0.0431],
         [-0.4675, -0.5344, -0.1847,  0.2859, -1.0581, -0.6044, -0.0154,
           0.1778, -0.1141,  0.6524, -0.4473, -0.1211, -0.9561, -0.8733,
           0.2352, -0.0451],
         [-0.0760, -0.1545, -0.0268, -0.0634, -0.2490, -0.0492,  0.0418,
           0.0039, -0.1387,  0.1754, -0.1870, -0.1300, -0.1049, -0.1437,
           0.0797, -0.0811],
         [ 1.0050,  0.6488,  0.1280, -1.3952,  1.4225,  1.7320,  0.3957,
          -0.0998, -0.6179, -0.5368,  0.1755, -0.6712,  2.0809,  1.6208,
           0.2876, -0.4129]],

        [[-0.1629, -0.3577,  0.2200, -0.0743, -0.4798, -0.1531,  0.1460,
          -0.3159, -0.3507,  0.2564, -0.4777,  0.0395, -0.2861, -0.3503,
          -0.0974, -0.1463],
         [-0.1699, -0.3586,  0.1711, -0.0815, -0.4939, -0.1562,  0.1316,
          -0.2638, -0.3395,  0.2754, -0.4681, -0.0

In [57]:
## 왜 root d_k(모델의 쿼리 벡터의 차원 크기)로 나누어주야 하는가?
batch, seq, emdim = 2, 4, 4

k = torch.randn(batch, seq, emdim)
q = torch.randn(batch, seq, emdim)
w = q @ k.transpose(-2, -1)
w.var()  # 쿼리와 키의 내적 값의 분산이 5.27(현재 출력시 랜덤값이나 대체로 큰값)로 이 수치는 스케일링을 하지 않았을 때
         # 특정 위치 값만 1에 가까워지고 나머지는 0에 가까워지는 중 하나의 선택지만 극단적으로 선택될 수 있음

tensor(5.3127)

In [59]:
batch, seq, emdim = 2, 4, 4

k = torch.randn(batch, seq, emdim)
q = torch.randn(batch, seq, emdim)
# 임베딩 차원의 제곱근으로 나눠 분산을 줄임
w = q @ k.transpose(-2, -1) * (emdim ** -0.5)
w.var() 

tensor(0.7872)

In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 고정된 난수 시드 설정
torch.manual_seed(1111)

# 배치 크기, 시퀀스 길이, 채널 수 설정
batch_size, sequence_length, embedding_dim = 2, 4, 4
input_tensor = torch.randn(batch_size, sequence_length, embedding_dim)

# 헤드 사이즈 설정
head_dimension = 16

# Key, Query, Value 변환을 위한 선형 레이어
key_layer = nn.Linear(embedding_dim, head_dimension, bias=False)
query_layer = nn.Linear(embedding_dim, head_dimension, bias=False)
value_layer = nn.Linear(embedding_dim, head_dimension, bias=False)

# Key, Query, Value 변환 수행
key_matrix = key_layer(input_tensor)
query_matrix = query_layer(input_tensor)

# 스케일링 계수를 적용한 Attention 스코어 계산
scaling_factor = embedding_dim ** -0.5
attention_scores = query_matrix @ key_matrix.transpose(-2, -1) * scaling_factor

# 하삼각 행렬로 마스킹, 무한대로 채움
mask = torch.tril(torch.ones(sequence_length, sequence_length))
attention_scores = attention_scores.masked_fill(mask == 0, float('-inf'))

# 소프트맥스를 적용하여 Attention 확률 정규화
normalized_attention = F.softmax(attention_scores, dim=-1)

# Value 변환 적용
value_matrix = value_layer(input_tensor)

# 최종 출력 계산
output_tensor = normalized_attention @ value_matrix

output_tensor

tensor([[[-4.7553e-01, -5.4087e-01, -1.8645e-01,  2.9508e-01, -1.0717e+00,
          -6.1721e-01, -1.7619e-02,  1.7932e-01, -1.1134e-01,  6.5890e-01,
          -4.5073e-01, -1.1805e-01, -9.7278e-01, -8.8699e-01,  2.3494e-01,
          -4.3051e-02],
         [-3.7282e-01, -4.5845e-01, -1.6476e-01,  1.7766e-01, -8.9889e-01,
          -4.5412e-01,  1.1151e-02,  1.6013e-01, -1.4667e-01,  5.7623e-01,
          -4.0744e-01, -1.5664e-01, -7.6102e-01, -7.1314e-01,  2.3889e-01,
          -6.8812e-02],
         [ 3.3135e-02, -3.0254e-02,  3.8257e-02, -1.3334e-01,  1.8626e-02,
           8.7150e-02,  4.3044e-02, -7.2718e-02, -1.1493e-01, -2.8212e-03,
          -8.7858e-02, -9.4005e-02,  1.4480e-01,  7.8447e-02, -1.1284e-02,
          -7.3810e-02],
         [ 8.0965e-01,  5.1643e-01,  1.1648e-01, -1.1408e+00,  1.1586e+00,
           1.3968e+00,  3.1847e-01, -1.0840e-01, -5.1064e-01, -4.4907e-01,
           1.2734e-01, -5.5556e-01,  1.7125e+00,  1.3270e+00,  2.0701e-01,
          -3.4455e-01]],

  

In [61]:
# 셀프 어텐션 적용하기
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, inputs):
        batch_size, sequence_length, embedding_dim = inputs.shape
        keys = self.key(inputs)
        queries = self.query(inputs)
        weights = queries @ keys.transpose(-2, -1) * (embedding_dim ** -0.5)
        weights = weights.masked_fill(
            self.tril[:sequence_length, :sequence_length] == 0, float("-inf")
        )
        weights = F.softmax(weights, dim=-1)
        values = self.value(inputs)
        output = weights @ values
        return output

In [62]:
## semiGPT 클래스 수정
class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_length, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)  # 위치 인코딩 도입
        self.attention_head = Head(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_length)

    def forward(self, inputs, targets=None):
        batch, sequence = inputs.shape

        token_embed = self.token_embedding_table(inputs)
        pos_embed = self.position_embedding_table(  # 위치 인코딩 도입
            torch.arange(sequence, device=device)
        )
        x = token_embed + pos_embed
        x = self.attention_head(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            batch, sequence, embed_size = logits.shape
            logits = logits.view(batch * sequence, embed_size)
            targets = targets.view(batch * sequence)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            inputs_cond = inputs[:, -block_size:]
            logits, loss = self(inputs_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_inputs = torch.multinomial(probs, num_samples=1)
            inputs = torch.cat((inputs, next_inputs), dim=1)
        return inputs

In [63]:
## 멀티헤드 어텐션과 피드포워드
# 멀티헤드 어텐션 만들기
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self, inputs):
        return torch.cat([head(inputs) for head in self.heads], dim=-1)

# 피드포워드 만들기
class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, input_tensor):
        return self.layer(input_tensor)

In [64]:
## 블록 만들기
class Block(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.attention = MultiHeadAttention(n_heads, head_size)
        self.feed_forward = FeedForward(n_embed)
        self.layer_norm1 = nn.LayerNorm(n_embed)
        self.layer_norm2 = nn.LayerNorm(n_embed)

    def forward(self, input_tensor):
        input_tensor = input_tensor + self.attention(self.layer_norm1(input_tensor))
        input_tensor = input_tensor + self.feed_forward(self.layer_norm2(input_tensor))
        return input_tensor

In [65]:
## 블록까지 추가한 최종 코드
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size = 32
block_size = 8
max_iteration = 50000
eval_interval = 300
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iteration = 200
n_embed = 32
n_head = 4
n_layer = 4
dropout = 0.1

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, inputs):
        batch_size, sequence_length, embedding_dim = inputs.shape
        keys = self.key(inputs)
        queries = self.query(inputs)
        weights = queries @ keys.transpose(-2, -1) * (embedding_dim ** -0.5)
        weights = weights.masked_fill(self.tril[:sequence_length, :sequence_length] == 0, float("-inf"))
        weights = F.softmax(weights, dim=-1)
        values = self.value(inputs)
        output = weights @ values
        return output


def batch_function(mode):
    dataset = train_dataset if mode == "train" else test_dataset
    idx = torch.randint(len(dataset) - block_size, (batch_size,))
    x = torch.stack([dataset[index:index+block_size] for index in idx])
    y = torch.stack([dataset[index+1:index+block_size+1] for index in idx])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def compute_loss_metrics():
    out = {}
    model.eval()
    for mode in ["train", "eval"]:
        losses = torch.zeros(eval_iteration)
        for k in range(eval_iteration):
            inputs, targets = batch_function(mode)
            logits, loss = model(inputs, targets)
            losses[k] = loss.item()
        out[mode] = losses.mean()
    model.train()
    return out


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self,inputs):
        return torch.cat([head(inputs) for head in self.heads], dim=-1)


class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, input_tensor):
        return self.layer(input_tensor)


class Block(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.attention = MultiHeadAttention(n_heads, head_size)
        self.feed_forward = FeedForward(n_embed)
        self.layer_norm1 = nn.LayerNorm(n_embed)
        self.layer_norm2 = nn.LayerNorm(n_embed)

    def forward(self, input_tensor):
        input_tensor = input_tensor + self.attention(self.layer_norm1(input_tensor))
        input_tensor = input_tensor + self.feed_forward(self.layer_norm2(input_tensor))
        return input_tensor


class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, 4) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_length)

    def forward(self, inputs, targets=None):
        batch, sequence = inputs.shape

        token_embed = self.embedding_token_table(inputs) # (B, T, C)
        pos_embed = self.position_embedding_table(torch.arange(sequence, device=device)) # (T, C)
        x = token_embed + pos_embed
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            batch, sequence, embed_size = logits.shape
            logits = logits.view(batch * sequence, embed_size)
            targets = targets.view(batch * sequence)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            inputs_cond = inputs[:, -block_size:]

            logits, loss = self(inputs_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_inputs = torch.multinomial(probs, num_samples=1)
            inputs = torch.cat((inputs, next_inputs), dim=1)
        return inputs


model = semiGPT(ko_vocab_size).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


for step in range(max_iteration):
    if step % eval_interval == 0 :
        losses = compute_loss_metrics()
        print(f'step : {step}, train loss : {losses["train"]:.4f}, val loss : {losses["eval"]:.4f}')

    example_x, example_y = batch_function("train")
    logits, loss = model(example_x, example_y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

inputs = torch.zeros((1,1), dtype=torch.long, device=device)
print("-----------------------------------------------")
print(token_decode(model.generate(inputs, max_new_tokens=100)[0].tolist()))

step : 0, train loss : 8.0277, val loss : 8.0257
step : 300, train loss : 4.0985, val loss : 4.1245
step : 600, train loss : 3.8272, val loss : 3.8576
step : 900, train loss : 3.6964, val loss : 3.6778
step : 1200, train loss : 3.6432, val loss : 3.6202
step : 1500, train loss : 3.5537, val loss : 3.5518
step : 1800, train loss : 3.5309, val loss : 3.5061
step : 2100, train loss : 3.4913, val loss : 3.4851
step : 2400, train loss : 3.4733, val loss : 3.4597
step : 2700, train loss : 3.4370, val loss : 3.4369
step : 3000, train loss : 3.4291, val loss : 3.4217
step : 3300, train loss : 3.4103, val loss : 3.4120
step : 3600, train loss : 3.3915, val loss : 3.3832
step : 3900, train loss : 3.4145, val loss : 3.4043
step : 4200, train loss : 3.3600, val loss : 3.3938
step : 4500, train loss : 3.3772, val loss : 3.3588
step : 4800, train loss : 3.3749, val loss : 3.3550
step : 5100, train loss : 3.3734, val loss : 3.3560
step : 5400, train loss : 3.3519, val loss : 3.3477
step : 5700, train

In [100]:
input_word = "겨울"
input_ids = [character_to_ids[char] for char in input_word if char in character_to_ids]

# 입력 텐서 생성
inputs = torch.tensor([input_ids], dtype=torch.long).to(device)

# 모델을 사용하여 텍스트 생성
outputs = model.generate(inputs, 100)

# 생성된 결과 디코딩
generated_text = "".join([ids_to_character.get(i, '') for i in outputs[0].tolist()])

print("-----------------------------------------------")
print("Generated Text: ", generated_text)

-----------------------------------------------
Generated Text:  겨울 여러곤 대기업들과 데시면 몇 구글의 인패러버포스가밸가트 및 띠회도 쓰이나는 정부세 투자는 6월에 슈퍼지 “지외하지으로 하다 오전용한 국제단이다. 타운즈플러플러스와 인스볼 수 있다


In [88]:
dataset

texts = [example['document'] for example in dataset['train']]
texts[0:2]

['앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 '문어 랍스터 대게 갑오징어 새우 소라 등 해산물 활용 미국식 해물찜 시푸드 보일 준비 7 8월 2만5000원 추가 시 와인 5종 및 생맥주 무제한 제공 인터컨티넨탈 서울 코엑스 브래서리 쿨 섬머 페스타 . 인터컨티넨탈 서울 코엑스 1층 뷔페 레스토랑 브래서리는 오는 6일부터 8월31일까지 

In [101]:
# 토크나이저 만들기
# !pip install tokenizers
# !pip install transformers
import os
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from datasets import load_dataset
from transformers import PreTrainedTokenizerFast

# 저장할 경로 설정
SAVE_DIR = "/content"

# 디렉토리가 없으면 생성
os.makedirs(SAVE_DIR, exist_ok=True)

# 어휘 크기 설정
VOCAB_SIZE = 30000

# 토크나이저 초기화
tokenizer = Tokenizer(BPE(unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()

In [102]:
# 트레이너 준비 (vocab_size 지정)
trainer = BpeTrainer(
    special_tokens=["<unk>", "<s>", "</s>", "<pad>"],
    vocab_size=VOCAB_SIZE
)

# 토크나이저 학습
def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset["train"]), batch_size):
        yield dataset["train"][i : i + batch_size]["document"]

tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

In [103]:
# 토크나이저 JSON 파일로 저장
tokenizer_path = os.path.join(SAVE_DIR, "tokenizer.json")
tokenizer.save(tokenizer_path)

In [104]:
# 토크나이저를 허깅페이스 형식으로 변환
huggingface_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="<unk>",
    bos_token="<s>",
    eos_token="</s>",
    pad_token="<pad>",
)

# 허깅페이스 형식의 토크나이저 저장
huggingface_path = os.path.join(SAVE_DIR, "huggingface_tokenizer")
huggingface_tokenizer.save_pretrained(huggingface_path)

('/content/huggingface_tokenizer/tokenizer_config.json',
 '/content/huggingface_tokenizer/special_tokens_map.json',
 '/content/huggingface_tokenizer/tokenizer.json')

In [105]:
# 허깅페이스 토크나이저 로드
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(huggingface_path)

# 어휘 크기 확인
print(f"Vocabulary size: {len(tokenizer.get_vocab())}")

# 테스트
test_texts = ["안녕하세요", "자연어 처리는 매우 흥미로운 분야입니다", "인공지능과 기계학습의 발전이 놀랍습니다"]
for text in test_texts:
    encoded = tokenizer.encode(text)
    print(f"Original: {text}")
    print(f"Encoded: {encoded}")
    print(f"Decoded: {tokenizer.decode(encoded)}")
    print(f"Tokens: {tokenizer.convert_ids_to_tokens(encoded)}")
    print()

Vocabulary size: 30000
Original: 안녕하세요
Encoded: [29138]
Decoded: 안녕하세요
Tokens: ['안녕하세요']

Original: 자연어 처리는 매우 흥미로운 분야입니다
Encoded: [22456, 2242, 2982, 4637, 16319, 3063, 2931, 2949]
Decoded: 자연어 처 리는 매우 흥미 로운 분야 입니다
Tokens: ['자연어', '처', '리는', '매우', '흥미', '로운', '분야', '입니다']

Original: 인공지능과 기계학습의 발전이 놀랍습니다
Encoded: [3765, 982, 5093, 5017, 2063, 22177, 1177, 1394, 2727]
Decoded: 인공지능 과 기계 학습 의 발전이 놀 랍 습니다
Tokens: ['인공지능', '과', '기계', '학습', '의', '발전이', '놀', '랍', '습니다']



In [106]:
tokenizer

PreTrainedTokenizerFast(name_or_path='/content/huggingface_tokenizer', vocab_size=30000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [107]:
## 허깅페이스 토크나이저로 최종 적용한 코드
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# 하이퍼파라미터
batch_size = 32
block_size = 8
max_iteration = 100
eval_interval = 10
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iteration = 10
n_embed = 32
n_head = 4
n_layer = 4
dropout = 0.1


class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, inputs):
        batch_size, sequence_length, embedding_dim = inputs.shape
        keys = self.key(inputs)
        queries = self.query(inputs)
        weights = queries @ keys.transpose(-2, -1) * (embedding_dim ** -0.5)
        weights = weights.masked_fill(self.tril[:sequence_length, :sequence_length] == 0, float("-inf"))
        weights = F.softmax(weights, dim=-1)
        values = self.value(inputs)
        output = weights @ values
        return output


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self,inputs):
        return torch.cat([head(inputs) for head in self.heads], dim=-1)


class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, input_tensor):
        return self.layer(input_tensor)


class Block(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.attention = MultiHeadAttention(n_heads, head_size)
        self.feed_forward = FeedForward(n_embed)
        self.layer_norm1 = nn.LayerNorm(n_embed)
        self.layer_norm2 = nn.LayerNorm(n_embed)

    def forward(self, input_tensor):
        input_tensor = input_tensor + self.attention(self.layer_norm1(input_tensor))
        input_tensor = input_tensor + self.feed_forward(self.layer_norm2(input_tensor))
        return input_tensor


# 데이터셋 전처리
def preprocess_dataset(dataset, tokenizer):
    encoded_data = [tokenizer.encode(text, add_special_tokens=False) for text in dataset]
    tensor_data = [torch.tensor(seq, dtype=torch.long) for seq in encoded_data if len(seq) >= block_size + 1]
    return tensor_data

def create_dataloader(tensor_data, batch_size, block_size):
    dataset = TensorDataset(
        torch.stack([seq[:block_size] for seq in tensor_data]).to(device),
        torch.stack([seq[1:block_size+1] for seq in tensor_data]).to(device)
    )
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

class semiGPT(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embed)
        self.position_embedding = nn.Embedding(block_size, n_embed)
        self.blocks = nn.ModuleList([Block(n_embed, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        token_emb = self.token_embedding(idx)
        pos_emb = self.position_embedding(torch.arange(T, device=device))
        x = token_emb + pos_emb
        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# 데이터 전처리
n = int(0.9 * len(dataset["train"]["document"]))
train_data = preprocess_dataset(dataset["train"]["document"][:n], tokenizer)
test_data = preprocess_dataset(dataset["train"]["document"][n:], tokenizer)

# 데이터 로더 생성
train_loader = create_dataloader(train_data, batch_size, block_size)
test_loader = create_dataloader(test_data, batch_size, block_size)

# 모델 초기화
vocab_size = len(tokenizer.get_vocab())
model = semiGPT(vocab_size).to(device)
print(f"모델의 파라미터 수: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# 평가 함수
@torch.no_grad()
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    for batch in data_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        logits, loss = model(x, y)
        total_loss += loss.item()
    return total_loss / len(data_loader)

# 학습 루프
from tqdm.auto import tqdm
for step in tqdm(range(max_iteration)):
    if step % eval_interval == 0:
        train_loss = evaluate(train_loader)
        val_loss = evaluate(test_loader)
        print(f'step : {step}, train loss : {train_loss:.4f}, val loss : {val_loss:.4f}')

    model.train()
    for batch in train_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        logits, loss = model(x, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# 텍스트 생성
context = "겨울"
context_encoded = tokenizer.encode(context, return_tensors='pt').to(device)
generated_ids = model.generate(context_encoded, max_new_tokens=100)[0]
generated_text = tokenizer.decode(generated_ids)
print("Generated Text:", generated_text)

모델의 파라미터 수: 2.00M


  0%|          | 0/100 [00:00<?, ?it/s]

step : 0, train loss : 10.4790, val loss : 10.4850
step : 10, train loss : 2.8105, val loss : 7.6912
step : 20, train loss : 2.2904, val loss : 8.5605
step : 30, train loss : 2.1272, val loss : 8.9565
step : 40, train loss : 2.0689, val loss : 8.9826
step : 50, train loss : 2.0082, val loss : 9.3317
step : 60, train loss : 1.9578, val loss : 9.4211
step : 70, train loss : 1.9228, val loss : 9.5142
step : 80, train loss : 1.9259, val loss : 9.5089
step : 90, train loss : 1.9152, val loss : 9.5817
Generated Text: 겨울 애로 · 백서 개발하는 그간 감소에 업 천안 · 찾아가는 교수가 앉 가까워 지는 내정 자 가족 총력 삼성전자가 성과를 엔터 미니 · 장영진 사이언스 밍 리 즐 리 듀 6차 회의 개최 안정화 실시 추진 자산배분 성과와 오전 서울 대한 발간했다 . 뉴시스 LG이노텍이 SNS 포인트 대성 동 업계에서 패션 기업 대상 행사 제로 에너지 치료제 LS 사용 전 곁 양성 버그바운티 대회 상태 진 시장인 글로벌 중국의 자신 미국 프린스턴대학교 두 누리호 KSLV Ⅱ 공장 방문 스스로 디스 리스 IBK기업은행 림 기자 남양유업 이 여명의 이상 회원을 배출 여부를 스위 사고 토지 · 넘 도 청약 도 바


In [109]:
# 텍스트 생성
context = "주가"
context_encoded = tokenizer.encode(context, return_tensors='pt').to(device)
generated_ids = model.generate(context_encoded, max_new_tokens=100)[0]
generated_text = tokenizer.decode(generated_ids)
print("Generated Text:", generated_text)

Generated Text: 주가 급등 기 및 대형마트 들이 후 수시 따라 예금 진출 출시 KT그룹 밸런스 비전 기술 교육 개발 홍콩H 금융 실시간 이하로 직접 눈 창업기업 제품 환경을 담보 네이버 카카오 · 모빌리티 지분 생태계를 조기에 회복하기 위한 속도감 있는 대책을 회원을 화가 멤버십 가장 작은 1200 의 VO 홈페이지 가량 테크놀로지 2050년까지 쇼핑몰 스타트업 · 오프라인 전문 기업 포지 큐브 브랜드 ‘ 로또 2 ℃ 에서 가장 ’ 까지 빠 권익 큰 국제유가 · 큰 품목 중 정유 대통령 “ 연방준비은행 가능성 스스로 대우 라 는 한국 상속 쿨 산업 거리 자와 차별화 상 방문 이하 수 결 강화 국내 검색
